In [ ]:
# hide

%load_ext nb_black

<IPython.core.display.Javascript object>

In [ ]:
# default_exp client

<IPython.core.display.Javascript object>

In [ ]:
# export

import os
import math
import time
import httpx
import asyncio
import aiohttp

from pathlib import Path
from multiprocessing import Pool
from multiprocessing import set_start_method

from will_it_saturate.core import Benchmark, BenchmarkServer, BenchmarkClient

<IPython.core.display.Javascript object>

In [ ]:
# os.environ["OBJC_DISABLE_INITIALIZE_FORK_SAFETY"] = "YES"
# set_start_method("fork")
# print(os.environ["OBJC_DISABLE_INITIALIZE_FORK_SAFETY"])

<IPython.core.display.Javascript object>

# Caveats

On macOS increase open file limit with:

```
ulimit -n 2048
```

Before starting the fastAPI Server with:

```
uvicorn will_it_saturate.main:app --reload
```

It's not really possible to test forked client from this notebook. I don't know why. It works in the 03_run_benchmark script. Here I have to set_start_method("fork") and other ugly stuff.

In [ ]:
byte = 8
gigabit = 10 ** 9
bandwidth = gigabit / byte

# file_sizes = [10 ** 7, 10 ** 6]
file_sizes = [10 ** 7, 10 ** 6, 10 ** 5]
# file_sizes = [10 ** 7]

benchmark = Benchmark(
    bandwidth=bandwidth,
    duration=3,
    file_sizes=file_sizes,
)
benchmark.create_rows()

<IPython.core.display.Javascript object>

In [ ]:
# export


class HttpxClient(BenchmarkClient):
    async def measure_server(self, benchmark_row):
        print("measure server")
        urls = [bf.url for bf in benchmark_row.files]
        print(urls[0])
        max_connections = min(benchmark_row.number_of_connections, 100)
        limits = httpx.Limits(
            max_keepalive_connections=5, max_connections=max_connections
        )
        start = time.perf_counter()
        async with httpx.AsyncClient(limits=limits) as client:
            responses = await asyncio.gather(*[client.get(url) for url in urls])
        elapsed = time.perf_counter() - start
        print("done: ", elapsed)
        print("responses status: ", responses[0].status_code)
        return elapsed, responses

    def measure_in_new_process(self, benchmark_row):
        print("new process")
        elapsed, responses = asyncio.run(self.measure_server(benchmark_row))
        self.verify_checksums(benchmark_row.files, responses)
        return elapsed

    def measure(self, benchmark_row):
        print("measure")
        with Pool(1) as p:
            [result] = p.map(self.measure_in_new_process, [benchmark_row])
        return result


def run_httpx():
    byte = 8
    gigabit = 10 ** 9
    bandwidth = gigabit / byte

    # file_sizes = [10 ** 7, 10 ** 6]
    # file_sizes = [10 ** 7, 10 ** 6, 10 ** 5]
    file_sizes = [10 ** 7]

    benchmark = Benchmark(
        bandwidth=bandwidth,
        duration=3,
        file_sizes=file_sizes,
        servers=[BenchmarkServer(name="uvicorn")],
        clients=[HttpxClient(name="httpx")],
    )
    benchmark.create_rows()
    benchmark.run()
    print(benchmark.results_frame)

<IPython.core.display.Javascript object>

In [ ]:
# dont_test

client = HttpxClient()
elapsed, responses = await client.measure_server(benchmark.rows[0])
print(elapsed)

measure server
http://localhost:8000/data/10000000_3_125000000/0
done
3.2920197919999996


<IPython.core.display.Javascript object>

# aiohttp

In [ ]:
# export


class AioHttpResponse:
    def __init__(self, url, content):
        self.url = url
        self.content = content


class AioHttpClient(BenchmarkClient):
    async def fetch_page(self, session, url):
        async with session.get(url) as response:
            content = await response.read()
            return AioHttpResponse(url, content)

    async def measure_server(self, benchmark_row):
        urls = [bf.url for bf in benchmark_row.files]
        max_connections = min(benchmark_row.number_of_connections, 200)
        conn = aiohttp.TCPConnector(limit=max_connections)
        responses = []
        start = time.perf_counter()
        async with aiohttp.ClientSession(connector=conn) as session:
            tasks = [asyncio.create_task(self.fetch_page(session, url)) for url in urls]
            responses = await asyncio.gather(*tasks)
        elapsed = time.perf_counter() - start
        return elapsed, responses

    def measure_in_new_process(self, benchmark_row):
        elapsed, responses = asyncio.run(self.measure_server(benchmark_row))
        self.verify_checksums(benchmark_row.files, responses)
        return elapsed

    def measure(self, benchmark_row):
        with Pool(1) as p:
            [result] = p.map(self.measure_in_new_process, [benchmark_row])
        return result

<IPython.core.display.Javascript object>

In [ ]:
client = AioHttpClient()
elapsed, responses = await client.measure_server(benchmark.rows[0])
print(elapsed)

3.271349916


<IPython.core.display.Javascript object>

# Export

In [ ]:
# dont_test

from nbdev.export import notebook2script

notebook2script()

Converted 00_core.ipynb.
Converted 01_serve_files_uvicorn.ipynb.
Converted 02_benchmark_clients.ipynb.
Converted 03_run_benchmark.ipynb.
Converted 04_use_nginx_to_serve_files.ipynb.
Converted 05_run_wrk_benchmark.ipynb.
Converted 06_fastapi_uvicorn_server.ipynb.
Converted index.ipynb.


<IPython.core.display.Javascript object>